In [1]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())


/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages


In [2]:
import time
from icecream import ic
import requests,json 
from bs4 import BeautifulSoup       

In [3]:
headers = {
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Referer': 'https://xyq.cbg.163.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

def get_cookies(server_id):
    #all the following steps are to get a valid cookies
    session = requests.Session()
    session.get("https://xyq.cbg.163.com", headers=headers,)
    data={
          "act": "do_anon_auth",
          "server_id": server_id,
    }
    session.post("https://xyq.cbg.163.com/cgi-bin/login.py", data=data, headers=headers)
    return session.cookies 

cookies = get_cookies(79)

In [4]:
from pycbg.xyq.parser import Parser
from pycbg.xyq.object.xyqObject import *
p = Parser()

In [5]:
role_urls = []
for page in range(1):
    params = (
    ('act', 'search_role'),
    ('page', page),
    ('query_order', 'collect_num DESC')
)

    response = requests.get('https://xyq.cbg.163.com/cgi-bin/query.py', headers=headers, params=params, cookies=cookies)
    response.encoding = "GBK"
    bsobj = BeautifulSoup(response.text, "html.parser")
    role_info = bsobj.findAll("a",{"class":"soldImg"})
    role_urls.extend([info["href"] for info in role_info]) 

In [6]:
role_list = []
role_desc_list = []    
for index,url in enumerate(role_urls[:]):
    print(index)
    print(url)
    response = requests.get(url, cookies=cookies)
    print(response)
    response.encoding = "GBK"
    bsobj = BeautifulSoup(response.text, "html.parser")
    role_desc = bsobj.findAll("textarea", {"id":"equip_desc_value"})[0].text
    role_dict = eval(p.desc_proprocess(role_desc))
    role_desc_list.append(role_desc)
    role=p.parse_role(role_desc)
    role_list.append(role)

0
https://xyq.cbg.163.com/equip?s=79&eid=201807282000113-79-J872QGAUCAJZJ&equip_refer=33&view_loc=search_cond
<Response [200]>
1
https://xyq.cbg.163.com/equip?s=79&eid=201807261900113-79-7VDT6VPQJE1ZA&equip_refer=33&view_loc=search_cond
<Response [200]>
2
https://xyq.cbg.163.com/equip?s=79&eid=201806231800113-79-VTGMYJKDV5LQQ&equip_refer=33&view_loc=search_cond
<Response [200]>
3
https://xyq.cbg.163.com/equip?s=79&eid=201806080000113-79-QOOVLIRIFMYB1&equip_refer=33&view_loc=search_cond
<Response [200]>
4
https://xyq.cbg.163.com/equip?s=79&eid=201807241900113-79-9SCSFKMVWQLAS&equip_refer=33&view_loc=search_cond
<Response [200]>
5
https://xyq.cbg.163.com/equip?s=79&eid=201706171600113-79-7XXFOVVUP9UC2&equip_refer=33&view_loc=search_cond
<Response [200]>
17320
6
https://xyq.cbg.163.com/equip?s=79&eid=201804062000113-79-E4HC1XKDHZKPS&equip_refer=33&view_loc=search_cond
<Response [200]>
7
https://xyq.cbg.163.com/equip?s=79&eid=201808022300113-79-RLLCXGBPV3UIT&equip_refer=33&view_loc=search_

In [7]:
for equip in role.equip_list:
    print(equip.__dict__)

{'gemType': ['黑宝石'], 'specialEffect': [], 'meltDict': {}, 'level': 130, 'DEF': 70, 'HP': 305, 'durance': 254, 'gemLevel': 10, 'ultimateSkill': '笑里藏刀', 'hole': 5, 'totemCombo': '归元心法符石', 'creator': '狂神乱舞', 'typeID': 2955, 'lock': 0, 'lockNew': 0}
{'gemType': [], 'specialEffect': [], 'meltDict': {}, 'level': 120, 'accurateP': 469, 'DMG': 427, 'durance': 500, 'ultimateSkill': '笑里藏刀', 'creator': 'う晚秋强化打造', 'typeID': 1062, 'lock': 0, 'lockNew': 0}
{'gemType': ['月亮石'], 'specialEffect': [], 'meltDict': {}, 'level': 140, 'DEF': 138, 'MP': 154, 'durance': 460, 'gemLevel': 5, 'ultimateSkill': '破甲术', 'hole': 1, 'creator': '无±悔强化打造', 'typeID': 2566, 'lock': 0, 'lockNew': 0}
{'gemType': ['月亮石', '光芒石'], 'specialEffect': ['永不磨损'], 'meltDict': {}, 'level': 140, 'DEF': 263, 'HP': 200, 'durance': 445, 'gemLevel': 7, 'hole': 4, 'totemCombo': '凤舞九天', 'creator': '神器の制造强化打造', 'typeID': 2666, 'lock': 0, 'lockNew': 0}
{'gemType': [], 'specialEffect': ['永不磨损'], 'meltDict': {}, 'level': 130, 'accurateP': 531, '

In [8]:
for pet in role.pet_list:
    print(pet.__dict__)

{'skillList': ['573', '429', '408', '411', '424'], 'coreDict': {931: 5, 905: 5, 904: 5, 936: 5}, 'equip1': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x10c02ee80>, 'equip2': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x10c02edd8>, 'equip3': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x10c02ed68>, 'equip4': <pycbg.xyq.object.xyqObject.Equipment_pet object at 0x10c02ee48>, 'lock': 0, 'lockNew': 0, 'typeID': 102214, 'level': 180, 'DRNC': 6300, 'summon_color': 0, 'realColor': 0, 'APSkill': 429, 'AD_APTTD': 1255, 'DEF_APTTD': 1288, 'raw_growth': 1254, 'growth': 1.254, 'MP_APTTD': 2820, 'HP_APTTD': 4040, 'SPD_APTTD': 1227, 'MS_APTTD': 1088, 'used_yuanxiao': 0, 'used_ruyidan': 0, 'used_qianjinlu': 0, 'used_lianshou': 0, 'used_shuijinggao': 0, 'HP_P': 706, 'MP_P': 780, 'AD_P': 190, 'DEF_P': 213, 'SPD_P': 197, 'unallocated_P': 0, 'lingxing': 93, 'tmp_lingxing': 93, 'jinjie_cd': 0, 'left_qlxl': 0, 'coreName': '', 'coreID': 0}
{'skillList': ['316', '301', '305', '304'],

In [9]:
print(len(role_list))

15
